<a href="https://www.kaggle.com/code/skshmjn/bert-ner?scriptVersionId=214325775" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
!pip install transformers datasets tokenizers seqeval evaluate peft -q 

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [3]:
import datasets 
import numpy as np 
import torch 
import json
from transformers import pipeline
from evaluate import load
from transformers import BertTokenizerFast
from transformers import DataCollatorForTokenClassification , DataCollatorWithPadding
from transformers import AutoModelForTokenClassification , AutoTokenizer
from transformers import TrainingArguments, Trainer 
import os
os.environ["WANDB_DISABLED"] = "true"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
conll2003 = datasets.load_dataset("conll2003", trust_remote_code=True)
conll2003

README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [4]:
conll2003["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [13]:
ner_classes = conll2003["train"].features["ner_tags"].feature.names
ner_classes

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [6]:
example = conll2003["train"][345]
example['tokens'] ,[ner_classes[i] for i in example['ner_tags']]

(['SOCCER', '-', 'GLORIA', 'BISTRITA', 'BEAT', '2-1', 'F.C.', 'VALLETTA', '.'],
 ['O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'B-ORG', 'I-ORG', 'O'])

In [5]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
def tokenize_and_align_labels(examples, label_all_tokens=True): 

    # Tokenisation
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True) 
    labels = [] 

    # checking all the labels and adding label for word piece 
    for i, label in enumerate(examples["ner_tags"]):
        
        word_ids = tokenized_inputs.word_ids(batch_index=i) 
        previous_word_idx = None 
        label_ids = []
        
        for word_idx in word_ids: 
            if word_idx is None: 
               
                label_ids.append(0)
           
            elif word_idx != previous_word_idx:
               
                label_ids.append(label[word_idx]) 
            else: 
               
                label_ids.append(label[word_idx] if label_all_tokens else 0) 
                 
            previous_word_idx = word_idx 
        labels.append(label_ids) 
    tokenized_inputs["labels"] = labels 
    return tokenized_inputs 

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
result = tokenize_and_align_labels(conll2003['train'][345:346])
print(result)

In [ ]:
for token, label in zip(tokenizer.convert_ids_to_tokens(result["input_ids"][0]),result["labels"][0]): 
    print(f"{token:_<40} {label}") 

In [6]:
tokenized_datasets = conll2003.map(tokenize_and_align_labels, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["tokens","pos_tags", "chunk_tags", "ner_tags"])

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [8]:
tokenized_datasets['train'][0:3]

{'id': ['0', '1', '2'],
 'input_ids': [[101,
   7327,
   19164,
   2446,
   2655,
   2000,
   17757,
   2329,
   12559,
   1012,
   102],
  [101, 2848, 13934, 102],
  [101, 9371, 2727, 1011, 5511, 1011, 2570, 102]],
 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1]],
 'labels': [[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100],
  [-100, 1, 2, -100],
  [-100, 5, 0, 0, 0, 0, 0, -100]]}

In [7]:
data_collator = DataCollatorForTokenClassification(tokenizer) 
metric = load("seqeval") 
def compute_metrics(eval_preds): 
 
    pred_logits, labels = eval_preds 
    
    pred_logits = np.argmax(pred_logits, axis=2) 
    
    predictions = [ 
        [ner_classes[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100] 
        for prediction, label in zip(pred_logits, labels) 
    ] 
    
    true_labels = [ 
      [ner_classes[l] for (eval_preds, l) in zip(prediction, label) if l != -100] 
       for prediction, label in zip(pred_logits, labels) 
   ] 
    results = metric.compute(predictions=predictions, references=true_labels) 
    return { 
   "precision": results["overall_precision"], 
   "recall": results["overall_recall"], 
   "f1": results["overall_f1"], 
  "accuracy": results["overall_accuracy"], 
  } 

In [47]:
args = TrainingArguments( 
"test-ner",
evaluation_strategy = "epoch", 
learning_rate=2e-5, 
per_device_train_batch_size=64, 
per_device_eval_batch_size=64, 
num_train_epochs=5, 
weight_decay=0.01, 
) 

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [48]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=9).to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
trainer = Trainer( 
    model, 
    args, 
   train_dataset=tokenized_datasets["train"], 
   eval_dataset=tokenized_datasets["validation"], 
   data_collator=data_collator, 
   tokenizer=tokenizer, 
   compute_metrics=compute_metrics 
) 

/tmp/ipykernel_24/999771.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(tokenized_datasets["test"])

In [ ]:
model.save_pretrained("ner_model")
tokenizer.save_pretrained("tokenizer")

In [ ]:
id2label = {
    str(i): label for i,label in enumerate(ner_classes)
}
label2id = {
    label: str(i) for i,label in enumerate(ner_classes)
}

config = json.load(open("ner_model/config.json"))
config["id2label"] = id2label
config["label2id"] = label2id
json.dump(config, open("ner_model/config.json","w"))

In [ ]:
model_fine_tuned = AutoModelForTokenClassification.from_pretrained("ner_model").to(device)
nlp = pipeline("ner", model=model_fine_tuned, tokenizer=tokenizer)

example = "Narendra Damodardas Modi[a] (born 17 September 1950)[b] is an Indian politician who has served as Prime Minister of India since 2014. Modi was the chief minister of Gujarat from 2001 to 2014 and is the member of parliament (MP) for Varanasi. He is a member of the Bharatiya Janata Party (BJP) and of the Rashtriya Swayamsevak Sangh (RSS), a right-wing Hindu nationalist paramilitary volunteer organisation. He is the longest-serving prime minister outside the Indian National Congress."

ner_results = nlp(example)

print(ner_results)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!sudo apt-get install git-lfs

In [ ]:
trainer.push_to_hub()

## Quantisation

In [17]:
!pip install --upgrade transformers auto-round

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 81.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.2/214.2 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 70.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 78.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 MB 18.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 MB 37.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [14]:
model_fine_tuned = AutoModelForTokenClassification.from_pretrained("skshmjn/test-ner").to(device)
auto_round_fine_tuned = AutoModelForTokenClassification.from_pretrained("skshmjn/test-ner").to(device)
tokenizer = AutoTokenizer.from_pretrained('skshmjn/test-ner')

config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [20]:
from auto_round import AutoRound
bits, group_size, sym = 4, 128, True
autoround = AutoRound(auto_round_fine_tuned, tokenizer, bits=bits, group_size=group_size, batch_size=2, seqlen=512, sym=sym, gradient_accumulate_steps=4, device=device)
autoround.quantize()
auto_round_fine_tuned.save_pretrained("auto_round_fine_tuned")


2024-12-19 16:26:18 INFO autoround.py L230: using torch.float16 for quantization tuning
2024-12-19 16:26:18 INFO autoround.py L300: start to cache block inputs


README.md:   0%|          | 0.00/373 [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/921 [00:00<?, ?B/s]

(…)-00000-of-00001-4746b8785c874cc7.parquet:   0%|          | 0.00/33.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

2024-12-19 16:27:15 INFO autoround.py L305: caching done
Quantizing bert.encoder.layer.0:   0%|          | 0/12 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/autograd/graph.py:768: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/Context.cpp:185.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/opt/conda/lib/python3.10/site-packages/torch/autograd/graph.py:768: UserWarning: Me

(BertForTokenClassification(
   (bert): BertModel(
     (embeddings): BertEmbeddings(
       (word_embeddings): Embedding(30522, 768, padding_idx=0)
       (position_embeddings): Embedding(512, 768)
       (token_type_embeddings): Embedding(2, 768)
       (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
       (dropout): Dropout(p=0.1, inplace=False)
     )
     (encoder): BertEncoder(
       (layer): ModuleList(
         (0-11): 12 x BertLayer(
           (attention): BertAttention(
             (self): BertSdpaSelfAttention(
               (query): Linear(in_features=768, out_features=768, bias=True)
               (key): Linear(in_features=768, out_features=768, bias=True)
               (value): Linear(in_features=768, out_features=768, bias=True)
               (dropout): Dropout(p=0.1, inplace=False)
             )
             (output): BertSelfOutput(
               (dense): Linear(in_features=768, out_features=768, bias=True)
               (LayerNorm): Layer

In [47]:
auto_round_fine_tuned = AutoModelForTokenClassification.from_pretrained("auto_round_fine_tuned",torch_dtype=torch.float16)
print_size_of_model(auto_round_fine_tuned)

Size (KB): 217875.845


In [16]:
quantized_model = torch.quantization.quantize_dynamic(
    model_fine_tuned, {torch.nn.Linear}, dtype=torch.qint8,
).to('cpu')

In [38]:
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp_delme.p")
    print('Size (KB):', os.path.getsize("temp_delme.p")/1e3)
    os.remove('temp_delme.p')


In [ ]:
fine_tune_trainer = Trainer( 
    model_fine_tuned, 
    args, 
   train_dataset=tokenized_datasets["train"], 
   eval_dataset=tokenized_datasets["validation"], 
   data_collator=data_collator, 
   tokenizer=tokenizer, 
   compute_metrics=compute_metrics 
) 

quntised_trainer = Trainer( 
    quantized_model, 
    args, 
   train_dataset=tokenized_datasets["train"], 
   eval_dataset=tokenized_datasets["validation"], 
   data_collator=data_collator, 
   tokenizer=tokenizer, 
   compute_metrics=compute_metrics , 
) 

autoround_trainer = Trainer( 
    auto_round_fine_tuned, 
    args, 
   train_dataset=tokenized_datasets["train"], 
   eval_dataset=tokenized_datasets["validation"], 
   data_collator=data_collator, 
   tokenizer=tokenizer, 
   compute_metrics=compute_metrics 
) 

In [19]:
print_size_of_model(model_fine_tuned), fine_tune_trainer.evaluate(tokenized_datasets["test"])

Size (KB): 435672.965


(None,
 {'eval_loss': 0.10942327231168747,
  'eval_model_preparation_time': 0.0037,
  'eval_precision': 0.8816678313533659,
  'eval_recall': 0.8984096843104676,
  'eval_f1': 0.8899600282153775,
  'eval_accuracy': 0.9746219183758028,
  'eval_runtime': 296.4786,
  'eval_samples_per_second': 11.647,
  'eval_steps_per_second': 0.182})

In [20]:
print_size_of_model(quantized_model), quntised_trainer.evaluate(tokenized_datasets["test"])

Size (KB): 180905.751


(None,
 {'eval_loss': 0.18233591318130493,
  'eval_model_preparation_time': 0.0036,
  'eval_precision': 0.8597059556441565,
  'eval_recall': 0.8188938998338476,
  'eval_f1': 0.838803792851933,
  'eval_accuracy': 0.9648504937504316,
  'eval_runtime': 223.7837,
  'eval_samples_per_second': 15.43,
  'eval_steps_per_second': 0.241})

In [52]:
print_size_of_model(auto_round_fine_tuned), autoround_trainer.evaluate(tokenized_datasets["test"])

Size (KB): 217875.845


(None,
 {'eval_loss': 0.10924585163593292,
  'eval_model_preparation_time': 0.0046,
  'eval_precision': 0.8755230125523012,
  'eval_recall': 0.8940185141229527,
  'eval_f1': 0.8846741045214328,
  'eval_accuracy': 0.9735342863061943,
  'eval_runtime': 277.2059,
  'eval_samples_per_second': 12.456,
  'eval_steps_per_second': 0.195})

In [49]:
print_size_of_model(model_fine_tuned), print_size_of_model(quantized_model), print_size_of_model(auto_round_fine_tuned)

Size (KB): 435672.965
Size (KB): 180905.751
Size (KB): 217875.845


(None, None, None)

## LORA


## using standard bert again to fine tune and compared result to earlier fine tuning

In [23]:
from peft import LoraConfig, TaskType
from peft import get_peft_model

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=1, lora_alpha=1, lora_dropout=0.1
)

In [24]:
# tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")
model = AutoModelForTokenClassification.from_pretrained(
    'bert-base-uncased', num_labels = 9
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
peft_model = get_peft_model(model, lora_config)

/opt/conda/lib/python3.10/site-packages/peft/mapping.py:185: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'bert-base-uncased' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(


In [29]:
args = TrainingArguments( 
"lora-ner",
evaluation_strategy = "epoch", 
learning_rate=1e-3, 
per_device_train_batch_size=64, 
per_device_eval_batch_size=64, 
num_train_epochs=5, 
weight_decay=0.01
) 
trainer = Trainer( 
    model, 
    args, 
   train_dataset=tokenized_datasets["train"], 
   eval_dataset=tokenized_datasets["validation"], 
   data_collator=data_collator, 
   tokenizer=tokenizer, 
   compute_metrics=compute_metrics 
) 

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipykernel_23/152047694.py:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [30]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.115601,0.796979,0.820338,0.808490,0.965556
2,No log,0.091624,0.843955,0.865197,0.854444,0.972742
3,No log,0.077937,0.862764,0.885446,0.873958,0.976082
4,No log,0.074616,0.863542,0.888466,0.875827,0.976889
5,0.131500,0.073099,0.868504,0.891822,0.880009,0.977407


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=550, training_loss=0.12807010824030096, metrics={'train_runtime': 348.6842, 'train_samples_per_second': 201.343, 'train_steps_per_second': 1.577, 'total_flos': 2237676578175600.0, 'train_loss': 0.12807010824030096, 'epoch': 5.0})

In [32]:
trainer.evaluate(tokenized_datasets["test"])

{'eval_loss': 0.0965961217880249,
 'eval_precision': 0.8457775189283634,
 'eval_recall': 0.8617374792309518,
 'eval_f1': 0.853682911057551,
 'eval_accuracy': 0.9714484035168903,
 'eval_runtime': 9.1346,
 'eval_samples_per_second': 378.012,
 'eval_steps_per_second': 2.956,
 'epoch': 5.0}

In [35]:
model.save_pretrained("lora_ner_model")
tokenizer.save_pretrained("lora_ner_model")

('lora_ner_model/tokenizer_config.json',
 'lora_ner_model/special_tokens_map.json',
 'lora_ner_model/vocab.txt',
 'lora_ner_model/added_tokens.json',
 'lora_ner_model/tokenizer.json')

In [37]:
id2label = {
    str(i): label for i,label in enumerate(ner_classes)
}
label2id = {
    label: str(i) for i,label in enumerate(ner_classes)
}

config = json.load(open("/kaggle/working/lora_ner_model/adapter_config.json"))
config["id2label"] = id2label
config["label2id"] = label2id
json.dump(config, open("/kaggle/working/lora_ner_model/adapter_config.json","w"))

In [44]:
print_size_of_model(peft_model), trainer.evaluate(tokenized_datasets["test"])

Size (KB): 435913.143
